In [ ]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph
# https://blog.0x.org/0x-ecosystem-update-13/

In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os
from datetime import datetime
import numpy as np
pd.set_option('display.max_rows', 99999)
pd.set_option('display.max_columns', 99999)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def client(url):
    sample_transport = RequestsHTTPTransport(url = url,
                                             verify = True,
                                             retries = 3)
    
    client = Client(transport=sample_transport)
    return client

In [61]:
# interesting tokens on MATIC

 #https://polygon.fulcrum.trade/borrow/asset
lending_tokens_fulcrum = ['LINK', 'BZRX', 'AAVE', 'WBTC', 'USDC', 'USDT', 'WETH', 'WMATIC']

# https://creamfinance.substack.com/p/cream-communications-14
lending_tokens_aave = ['MATIC', 'DAI', 'USDC', 'USDT', 'WETH', 'WBTC', 'AAVE']
lending_tokens_cream = ['USDC', 'USDT', 'DAI', 'WMATIC', 'WETH', 'WBTC', 'LINK', 'SUSHI', 'CRV', 'QUICK']
remaining_interesting_tokens = ['FRAX', 'ibBTC']

interesting_tokens = list(set(lending_tokens_fulcrum +\
                              lending_tokens_aave +\
                              lending_tokens_cream +\
                              remaining_interesting_tokens))

interesting_tokens

['AAVE',
 'USDC',
 'FRAX',
 'MATIC',
 'USDT',
 'SUSHI',
 'CRV',
 'QUICK',
 'DAI',
 'WETH',
 'LINK',
 'ibBTC',
 'BZRX',
 'WBTC',
 'WMATIC']

## Quickswap

In [4]:
query = gql('''
query {
  uniswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sameepsi/quickswap02').execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

6305


In [5]:
try:
    df = pd.read_excel('pairdata_Quickswap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [6]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [7]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel('pairdata_Quickswap_MATIC.xlsx', index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,WMATIC-QUICK,WMATIC,QUICK
1,0x02e0e14d65175420fb12acd325a7be456bc22005,QUICK-DAI,QUICK,DAI
2,0x0554059d42e26f35cc958581c71fdfd92405d02f,WMATIC-AAVE,WMATIC,AAVE
3,0x0b3a76d33d93b2651dbf03b102eefa9500e89243,SUSHI-WMATIC,SUSHI,WMATIC
4,0x1bd06b96dd42ada85fdd0795f3b4a79db914add5,WETH-QUICK,WETH,QUICK


In [8]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [9]:
df_old = pd.read_csv("Quickswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-08 17:31:25.617431,8.719190e+08


In [10]:
df4.to_csv("Quickswap export.csv", index = False)

In [11]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']
df_combined.head(3)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-08 21:24:06.201775,8.722182e+08,2021-07-08 17:31:25.617431,8.719190e+08,0.000442,299223.046562,2.029120e+06
0x02e0e14d65175420fb12acd325a7be456bc22005,2021-07-08 21:24:06.398982,2.329145e+05,2021-07-08 17:31:25.821761,2.328409e+05,0.000442,73.693620,4.997385e+02
0x0554059d42e26f35cc958581c71fdfd92405d02f,2021-07-08 21:24:06.604305,4.394834e+05,2021-07-08 17:31:26.025899,4.391072e+05,0.000442,376.183555,2.551013e+03


In [12]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [13]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['dailyVolumeUSD','reserveUSD']] = df5[['dailyVolumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['dailyVolumeUSD'] * 0.003, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Quickswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 500), 3)
df5['APR yday Liq + 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 1000), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)
df5['APR yday Liq - 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 200), 3)
df5['APR yday Liq - 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 500), 3)
df5['APR yday Liq - 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 1000), 3)
df5['APR yday Liq - 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 2000), 3)

df5.head(2)

,dailyVolumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2.020198e+06,2021-07-08,2.378294e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,6060.595048,9.301,9.301,9.301,9.301,9.300,9.301,9.301,9.302,9.302
1,2.243389e+06,2021-07-07,2.557146e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,6730.166098,9.606,9.606,9.606,9.606,9.606,9.607,9.607,9.607,9.607


In [14]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,dailyVolumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xf6422b997c7f54d1c6a6e103bcb1499eea0a7046,5.576481e+06,2021-07-07,6.286321e+07,[Quickswap MATIC] WETH-USDT,16729.444405,9.714,9.714,9.713,9.713,9.713,9.714,9.714,9.714,9.714
0xfba51e1ea2d46c4d5e98f27e1f5333969d2222d7,2.979077e-01,2021-07-07,1.161333e+01,[Quickswap MATIC] LINK-DAI,0.000894,2.810,0.154,0.064,0.032,0.016,-0.173,-0.067,-0.033,-0.016


In [15]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x019ba0325f1988213d448b3472fa1cf8d07618d7,8.531830,8.531758,8.531651,8.531471,8.531113,8.531902,8.532009,8.532189,8.532548,2.378294e+07
0x02e0e14d65175420fb12acd325a7be456bc22005,6.658903,6.486052,6.242972,5.875946,5.257739,6.841218,7.134212,7.682591,9.078203,7.504818e+03


In [16]:

df_final_quickswap = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                             
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'dailyVolumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_quickswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                            
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                            
                              'APR timedelta Liq - 2000',                            
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',                                            
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',                                            
                              'APR yday Liq - 2000',                              
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_quickswap.head(1)

# df_final_quickswap_stylized = df_final_quickswap.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_quickswap_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Quickswap MATIC] WMATIC-USDT,2021-07-07,75.491028,75.474192,75.448952,75.406923,75.323004,75.507871,75.533151,75.575321,75.659803,40.351,40.343,40.332,40.312,40.274,40.359,40.371,40.39,40.429,896577.240178,1.037774e+06,382424.261618


In [17]:
# dfi.export(df_final_quickswap_stylized, 'Quickswap MATIC timedelta.png')
# os.system('xdg-open "Quickswap MATIC timedelta.png"')

## Sushiswap MATIC

In [18]:
# sushiswap_matic = 'https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange'

# sample_transport=RequestsHTTPTransport(url = sushiswap_matic,
#                                        verify = True,
#                                        retries = 3)

# client = Client(transport=sample_transport)

In [19]:
query = gql('''
query {
  factories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

1356


In [20]:
try:
    df = pd.read_excel('pairdata_SushiSwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [21]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [22]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_SushiSwap_MATIC.xlsx", index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,WMATIC-CRV,WMATIC,CRV
1,0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,SUSHI-DAI,SUSHI,DAI
2,0x264a05020b491ba1969a325b0a474a864bb0d93b,FRAX-USDT,FRAX,USDT
3,0x2813d43463c374a680f235c428fb1d7f08de0b69,WETH-AAVE,WETH,AAVE
4,0x29bb3ddf61c831adda06439740b340ae48606da6,WBTC-QUICK,WBTC,QUICK


In [23]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]
df4.head(1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

,id,timestamp,volumeUSD
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-08 21:26:41.080011,1137.919897


In [24]:
df_old = pd.read_csv("SushiSwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-08 17:33:53.387035,1135.85706


In [25]:
df4.to_csv("SushiSwap export.csv", index = False)

In [26]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-08 21:26:41.080011,1137.919897,2021-07-08 17:33:53.387035,1135.857060,0.000443,2.062837,13.981589
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,2021-07-08 21:26:41.287364,10099.558858,2021-07-08 17:33:53.584020,10095.724917,0.000443,3.833940,25.985834


In [27]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pair:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    volumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

In [28]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['volumeUSD','reserveUSD']] = df5[['volumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['volumeUSD'] * 0.003, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Sushiswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 500), 3)
df5['APR yday Liq + 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 1000), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)
df5['APR yday Liq - 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 200), 3)
df5['APR yday Liq - 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 500), 3)
df5['APR yday Liq - 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 1000), 3)
df5['APR yday Liq - 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 2000), 3)

df5.head(2)

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2021-07-08,567.692396,26.041550,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,0.078125,5.023,3.714,2.671,1.819,1.111,7.755,42.125,-6.596,-1.991
1,2021-07-07,610.566989,50.082764,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,0.150248,8.982,6.766,4.938,3.405,2.101,13.357,49.599,-14.082,-3.947


In [29]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,date,reserveUSD,volumeUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0x396e655c309676caf0acf4607a868e0cded876db,2021-07-07,3.197183e+06,198730.020505,[Sushiswap MATIC] CRV-WETH,596.190062,6.806,6.806,6.805,6.804,6.802,6.807,6.807,6.808,6.811
0xb5846453b67d0b4b4ce655930cf6e4129f4416d7,2021-07-07,1.030779e+06,460029.012239,[Sushiswap MATIC] SUSHI-WETH,1380.087037,48.869,48.860,48.845,48.822,48.774,48.879,48.893,48.917,48.964


In [30]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2.462881,1.821249,1.309515,0.891858,0.544520,3.802523,20.654593,-3.234176,-0.976158,567.692396
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,5.307540,3.768235,2.625887,1.744481,1.043775,8.972941,-249.916433,-5.091290,-1.720463,489.602191


In [31]:
df_final_sushiswap_matic = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                             
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                                'APR timedelta Liq + 500',
                                'APR timedelta Liq + 1000',                                            
                                'APR timedelta Liq + 2000',
                                'APR timedelta Liq - 200',
                                'APR timedelta Liq - 500',
                                'APR timedelta Liq - 1000',                                            
                                'APR timedelta Liq - 2000',                           
                              'APR yday',
                             'APR yday Liq + 200',
                                'APR yday Liq + 500',
                                'APR yday Liq + 1000',                                            
                                'APR yday Liq + 2000',
                                'APR yday Liq - 200',
                                'APR yday Liq - 500',
                                'APR yday Liq - 1000',                                            
                                'APR yday Liq - 2000',                          
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_matic.head(1)

# df_final_sushiswap_matic_stylized = df_final_sushiswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_sushiswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Sushiswap MATIC] SUSHI-WMATIC,2021-07-07,105.980858,105.911625,105.807945,105.635596,105.292577,106.050181,106.154337,106.328384,106.678197,45.078,45.05,45.007,44.935,44.793,45.107,45.15,45.222,45.367,305957.969603,313921.810101,129233.392716


In [32]:
# dfi.export(df_final_stylized_sushi, 'SushiSwap MATIC timedelta.png')
# os.system('xdg-open "SushiSwap MATIC timedelta.png"')

## Honeyswap Polygon

In [33]:
query = gql('''
query {
  honeyswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(query)
number_of_pairs = int(response['honeyswapFactories'][0]['pairCount'])
print(number_of_pairs)

147


In [34]:
try:
    df = pd.read_excel('pairdata_HoneySwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [35]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [36]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_HoneySwap_MATIC.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
0,0x0c787944946d22922c9f41c477cc539700d35bb2,WETH-AAVE,WETH,AAVE
1,0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,WETH-USDT,WETH,USDT


In [37]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

,id,timestamp,volumeUSD
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-08 21:27:00.738151,72842.966273


In [38]:
df_old = pd.read_csv("HoneySwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-08 17:34:12.849811,69602.257565


In [39]:
df_timedelta.to_csv("HoneySwap export.csv", index = False)

In [40]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.0025 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-08 21:27:00.738151,72842.966273,2021-07-08 17:34:12.849811,69602.257565,0.000443,3240.708707,18303.928024
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,2021-07-08 21:27:00.956601,141254.188980,2021-07-08 17:34:13.066664,140707.639472,0.000443,546.549509,3086.979616


In [41]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

In [42]:
df_last_two_days2 = df_last_two_days.merge(df_filtered,
                             left_on = 'id',
                             right_on = 'id',
                             how = 'left',
                             validate = 'many_to_one')
df_last_two_days2[['volumeUSD','reserveUSD']] = df_last_two_days2[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days2['date'] = pd.to_datetime(df_last_two_days2['date'], unit='s').dt.date
df_last_two_days2['fee'] = round(df_last_two_days2['volumeUSD'] * 0.0025, 6)
df_last_two_days2['APR yday'] = round(100 * df_last_two_days2['fee'] * 365 / df_last_two_days2['reserveUSD'], 3)
df_last_two_days2['Pair'] = '[Honeyswap MATIC] ' + df_last_two_days2['Pair']
df_last_two_days2['APR yday Liq + 200'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 200), 3)
df_last_two_days2['APR yday Liq + 500'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 500), 3) 
df_last_two_days2['APR yday Liq + 1000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 1000), 3)                                                  
df_last_two_days2['APR yday Liq + 2000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 2000), 3)
df_last_two_days2['APR yday Liq - 200'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 200), 3)
df_last_two_days2['APR yday Liq - 500'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 500), 3)
df_last_two_days2['APR yday Liq - 1000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 1000), 3)                                                 
df_last_two_days2['APR yday Liq - 2000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 2000), 3)                                                 

df_last_two_days2.head(2)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,9753.355913,2021-07-08,167593.129471,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,24.383390,5.310,5.304,5.295,5.279,5.248,5.317,5.326,5.342,5.375
1,5917.458919,2021-07-07,180749.818634,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,14.793647,2.987,2.984,2.979,2.971,2.955,2.991,2.996,3.004,3.021


In [43]:
df_yesterday = df_last_two_days2.sort_values(by = 'date',
                                             ascending = False)\
                                .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                .reset_index(drop = True)\
                                .drop(columns = ['Token 0', 'Token 1'])\
                                .set_index('id')

df_yesterday.head(2)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0x96aa3911e89da7b6d8b4a2b796f3000987d23cf2,1.968537,2021-07-07,898.590284,[Honeyswap MATIC] DAI-USDT,0.004921,0.200,0.163,0.128,0.095,0.062,0.257,0.451,-1.771,-0.163
0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc,2.554731,2021-07-07,20.002205,[Honeyswap MATIC] WMATIC-CRV,0.006387,11.655,1.060,0.448,0.229,0.115,-1.295,-0.486,-0.238,-0.118


In [44]:
df_combined2 = pd.concat([df_combined,
                          df_last_two_days2.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',
                             'APR timedelta Liq - 2000',                              
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x0c787944946d22922c9f41c477cc539700d35bb2,10.921646,10.908628,10.889159,10.856865,10.792848,10.934695,10.954327,10.987205,11.053555,167593.129471
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,4.133517,4.122477,4.106027,4.078900,4.025707,4.144616,4.161378,4.189617,4.247260,74681.672413


In [45]:
df_final_honeyswap_matic = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                                 
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_honeyswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                    
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                    
                              'APR timedelta Liq - 2000',                                    
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',
                              'APR yday Liq - 2000',                                     
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_honeyswap_matic.head(1)

# df_final_honeyswap_matic_stylized = df_final_honeyswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_honeyswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Honeyswap MATIC] WMATIC-WETH,2021-07-07,19.910277,19.899209,19.882631,19.855061,19.800151,19.921357,19.938,19.9658,20.021635,6.019,6.017,6.013,6.006,5.992,6.022,6.026,6.033,6.046,359590.859615,447783.096612,29537.669245


## DoDoSwap Polygon

In [88]:
# client_exe = client('https://api.thegraph.com/subgraphs/name/dodoex/dodoex-v2-polygon')
# # https://thegraph.com/legacy-explorer/subgraph/dodoex/dodoex-v2-polygon
# fee = 0.0025

In [87]:
# query = gql('''
# query {
#   dodoZoos {
#     pairCount
#   }
# }
# ''')

# response = client_exe.execute(query)
# number_of_pairs = int(response['dodoZoos'][0]['pairCount'])
# print(number_of_pairs)

In [86]:
# # try:
# #     df = pd.read_excel('pairdata_HoneySwap_MATIC.xlsx')
# # except:
# df = pd.DataFrame([])
# response_dict = {}
# response_list = []


# for i in range(number_of_pairs):
#     if i % 100 == 0:
#         querystring = '''
#               query {
#               pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
#                       baseSymbol
#                       quoteSymbol
#                       id
#                   }
#                 }
#                 '''
#         try:
#             response = client_exe.execute(gql(querystring))
#             df = df.append(pd.DataFrame.from_dict(response['pairs']))
#             print(len(df))
#         except:
#             pass

In [85]:
# df = df.dropna()


# df['Pair'] = df['baseSymbol'] + '-' + df['quoteSymbol']
# df.columns = ['Token 0', 'id', 'Token 1', 'Pair']
# df = df[['id','Pair', 'Token 0', 'Token 1']]
# df.head(2)

In [84]:
# criteria = df['Token 0'].isin(interesting_tokens) & \
#            df['Token 1'].isin(interesting_tokens)

# df_filtered = df.loc[criteria,:]
# df_filtered.to_excel("pairdata_DoDoSwap_MATIC.xlsx", index = False)
# df_filtered.head(2)

In [83]:
# df_timedelta = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')

#     querystring = '''
#                   query {
#       pairs(where: {id:"''' + id + '''"}){
#         volumeUSD
#       }
#     }'''
#     response = client_exe.execute(gql(querystring))

#     df1 = pd.DataFrame.from_dict(response['pairs'])
#     df1['id'] = id
#     df1['timestamp'] = datetime.now()
#     df1['volumeUSD'] = df1['volumeUSD'].astype(float)
#     df_timedelta = df_timedelta.append(df1)
    
# df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
# df_timedelta.head(1)

In [82]:
# df_old = pd.read_csv("DoDoSwap export.csv",
#                      usecols = ['id', 'timestamp', 'volumeUSD'])
# df_old = df_old[['id', 'timestamp', 'volumeUSD']]
# df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
# df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
# df_old.head(1)

In [81]:
# df_timedelta.to_csv("DoDoSwap export.csv", index = False)

In [80]:
# df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
# df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
# df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
# df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.0025 / df_combined['timedelta']
# df_combined.head(2)

In [79]:
# df_last_two_days = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')
        
#     querystring = '''
#               query {
#   pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
#     date
#     volumeUSD
#     baseUsdPrice
#     quoteUsdPrice
#     baseLpTokenTotalSupply
#     quoteLpTokenTotalSupply
#   }
# }'''
#     response = client_exe.execute(gql(querystring))
#     response
    
#     df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
#     df1['id'] = id
#     df_last_two_days = df_last_two_days.append(df1)
    
# df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

## Balancer Polygon

In [46]:
# client_exe = client('https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-polygon-v2')
# #https://thegraph.com/explorer/subgraph/balancer-labs/balancer-polygon-v2
# fee = 0.0025

In [47]:
# query = gql('''
#             query {
#                   balancers {
#                     poolCount
#                   }
#                 }
#             ''')

# response = client_exe.execute(query)
# number_of_pairs = int(response['balancers'][0]['poolCount'])
# print(number_of_pairs)

In [48]:
# # make a list of ids and pairs on balancer

# df = pd.DataFrame([])
# response_dict = {}
# response_list = []


# for i in range(number_of_pairs):
#     if i % 100 == 0:
#         querystring = '''
#               query {
#               pools(first: '''+str(100)+' skip: '+str(i)+'''){
#                       tokens{symbol}
#                       id
#                   }
#                 }
#                 '''
#         try:
#             response = client_exe.execute(gql(querystring))
#             df = df.append(pd.DataFrame.from_dict(response['pools']))
#             print(len(df))
#         except:
#             pass
    
# df['tokens'] = df['tokens'].astype(str).str.replace("{'symbol': ",'', regex= True)\
#                                            .replace("}",'', regex= True)\
#                                            .replace("]",'', regex= True)\
#                                            .replace('\[', '', regex = True)\
#                                            .replace("'",'', regex= True)\
#                                            .replace(" ",'', regex= True)

# number_different_tokens = len(df['tokens'].str.split(',', expand = True).columns)
# column_list = []

# for token_column in range(number_different_tokens):
#     column_list.append('Token' + str(token_column + 1))
    
# df[column_list] = df['tokens'].str.split(',', expand = True)
# df.replace()

In [49]:
# # make a list of all tokens on Balancer that are not in my interesting tokens

# df_tokens = pd.DataFrame([])

# for column in column_list:
#     df_tokenpart = df[[column]]
#     df_tokenpart = df_tokenpart.rename(columns = {column:'Token'})
#     df_tokens = df_tokens.append(df_tokenpart)
    
# all_balancer_tokens = df_tokens.drop_duplicates().dropna().sort_values(by = 'Token')['Token'].unique().tolist()
# tokens_to_avoid = set(all_balancer_tokens) - set(interesting_tokens)

In [50]:
# # filter out the pairs on balancer that contain tokens to avoid
# df_filtered = df.copy()

# for column in column_list:
#     df_filtered = df_filtered.loc[~df[column].isin(tokens_to_avoid)]
    
# df_filtered = df_filtered[['id', 'tokens']]
# df_filtered

In [51]:
# df_timedelta = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')

#     querystring = '''
#                   query {
#       pools(where: {id:"''' + id + '''"}){
#         totalSwapVolume
#       }
#     }'''
#     response = client_exe.execute(gql(querystring))

#     df1 = pd.DataFrame.from_dict(response['pools'])
#     df1['id'] = id
#     df1['timestamp'] = datetime.now()
#     df1['volumeUSD'] = df1['totalSwapVolume'].astype(float)
#     df_timedelta = df_timedelta.append(df1)
    
# df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
# df_timedelta.head(1)

In [52]:
# df_old = pd.read_csv("Balancer export MATIC.csv",
#                      usecols = ['id', 'timestamp', 'volumeUSD'])
# df_old = df_old[['id', 'timestamp', 'volumeUSD']]
# df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
# df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
# df_old.head(1)

In [53]:
# df_timedelta.to_csv("Balancer export MATIC.csv", index = False)

In [54]:
# df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
# df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
# df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
# df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

# df_combined.head(2)

In [55]:
# querystring = """
#               query {
#                     poolSnapshots(orderBy: timestamp, orderDirection: desc){
#                       timestamp
#                     }
#                     }
#                 """

# response = client_exe.execute(gql(querystring))

# df_time = pd.DataFrame.from_dict(response['poolSnapshots'])\
#                       .drop_duplicates()\
#                       .sort_values(by = 'timestamp', ascending = False).head(2)

# df_time['timestamp'].values.tolist()

In [56]:
# {
#   pools{
#     id
#     totalSwapVolume
#     totalLiquidity
#   }
# }

In [57]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   amounts
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
# }
# }
# """

In [58]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
#   pool{historicalValues(orderBy: block, orderDirection: desc first: 2){
#     poolLiquidity
#     poolId {
#       id
#     }
#     block
#     id
#   }}
# }
# }
# """

In [59]:
# df_last_two_days = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')
        
#     querystring = '''
#               query {
#   pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
#     date
#     dailyVolumeUSD
#     reserveUSD
#   }
# }'''
#     response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
#     response
    
#     df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
#     df1['id'] = id
#     df_last_two_days = df_last_two_days.append(df1)
    
# df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

## FTM settings

In [60]:
lending_tokens_cream = ['USDC', 'WFTM', 'DAI', 'ETH', 'BTC', 'LINK', 'SUSHI', 'YFI', 'SNX', 'BAND', 'AAVE', 'COVER', 'CREAM', 'HEGIC']
remaining_interesting_tokens = ['WFTM', 'FTM', 'USDC', 'DAI', 'ETH', 'fUSDT', 'FETH', 'WBTC', 'FRAX', 'FRAX', 'ibBTC']

interesting_tokens = list(set(lending_tokens_cream +\
                              remaining_interesting_tokens))

## SpiritSwap

In [ ]:
query = gql('''
query {
  spiritswapFactories{
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(query)
number_of_pairs = int(response['spiritswapFactories'][0]['pairCount'])
print(number_of_pairs)

In [ ]:
try:
    df = pd.read_excel('pairdata_Spiritswap_FTM.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0{symbol}
                          token1{symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [ ]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [ ]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_Spiritswap_FTM.xlsx", index = False)
df_filtered.head(2)

In [ ]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

In [ ]:
df_old = pd.read_csv("Spiritswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

In [ ]:
df_timedelta.to_csv("Spiritswap export FTM.csv", index = False)

In [ ]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']

df_combined.head(2)

In [ ]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

In [ ]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * 0.003, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpiritSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

In [ ]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

In [ ]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

In [ ]:
# df_final_spiritswap = pd.concat([df_combined2, df_yesterday],
#                                  axis = 1)[['Pair',
#                                             'date',
#                                             'APR timedelta',
#                                             'APR timedelta Liq + 200',
#                                             'APR timedelta Liq + 500',
#                                             'APR timedelta Liq + 1000',                                            
#                                             'APR timedelta Liq + 2000',
#                                             'APR timedelta Liq - 200',
#                                             'APR timedelta Liq - 500',
#                                             'APR timedelta Liq - 1000',                                            
#                                             'APR timedelta Liq - 2000',                                             
#                                             'APR yday',
#                                             'APR yday Liq + 200',
#                                             'APR yday Liq + 500',
#                                             'APR yday Liq + 1000',                                            
#                                             'APR yday Liq + 2000',
#                                             'APR yday Liq - 200',
#                                             'APR yday Liq - 500',
#                                             'APR yday Liq - 1000',                                            
#                                             'APR yday Liq - 2000',                                            
#                                             'reserveUSD',
#                                             'volumeUSD']].set_index('Pair')\
#                        .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

# df_final_spiritswap.columns = ['date',
#                               'APR timedelta',
#                                 'APR timedelta',
#                                 'APR timedelta Liq + 200',
#                                 'APR timedelta Liq + 500',
#                                 'APR timedelta Liq + 1000',                                            
#                                 'APR timedelta Liq + 2000',
#                                 'APR timedelta Liq - 200',
#                                 'APR timedelta Liq - 500',
#                                 'APR timedelta Liq - 1000',                                            
#                                 'APR timedelta Liq - 2000',     
#                               'APR yday',
#                                 'APR yday Liq + 200',
#                                 'APR yday Liq + 500',
#                                 'APR yday Liq + 1000',                                            
#                                 'APR yday Liq + 2000',
#                                 'APR yday Liq - 200',
#                                 'APR yday Liq - 500',
#                                 'APR yday Liq - 1000',                                            
#                                 'APR yday Liq - 2000', 
#                               'Liquidity timedelta',
#                               'Liquidity yday',
#                               'Volume yday']

# df_final_spiritswap.head(1)

df_final_spiritswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                                 
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spiritswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                    
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                    
                              'APR timedelta Liq - 2000',                                    
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',
                              'APR yday Liq - 2000',                                     
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

## SpookySwap

In [ ]:
client_exe = client('https://api.thegraph.com/subgraphs/name/eerieeight/spooky-swap-exchange')
fee = 0.002

In [ ]:
query = gql('''
            query {uniswapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

In [ ]:
# try:
#     df = pd.read_excel('pairdata_Spookyswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []

for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
          query {
          pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                  token0 {symbol}
                  token1 {symbol}
                  id
              }
            }
            '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass

In [ ]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

In [ ]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

In [ ]:
df_old = pd.read_csv("Spookyswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

In [ ]:
df_timedelta.to_csv("Spookyswap export.csv", index = False)

In [ ]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

In [ ]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

In [ ]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpookySwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

In [ ]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

In [ ]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

In [ ]:
df_final_spookyswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spookyswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                               
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                               
                              'APR timedelta Liq - 2000',                               
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',                               
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',                               
                              'APR yday Liq - 2000',                               
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_spookyswap.head(1)

## SushiSwap FTM

In [ ]:
client_exe = client('https://api.thegraph.com/subgraphs/name/sushiswap/fantom-exchange')
fee = 0.003

In [ ]:
query = gql('''
            query {factories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

In [ ]:
# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

In [ ]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

In [ ]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

In [ ]:
df_old = pd.read_csv("Sushiswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

In [ ]:
df_timedelta.to_csv("Sushiswap export FTM.csv", index = False)

In [ ]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

In [ ]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pair:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            volumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

In [ ]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SushiSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

In [ ]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

In [ ]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

In [ ]:
df_final_sushiswap_ftm = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_ftm.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_ftm.head(1)

## WakaSwap FTM

In [ ]:
client_exe = client('https://api.thegraph.com/subgraphs/name/waka-finance/waka-graph') # https://thegraph.com/explorer/subgraph/waka-finance/waka-graph
fee = 0.003

query = gql('''
            query {wakaSwapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['wakaSwapFactories'][0]['pairCount'])
print(number_of_pairs)

# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

In [ ]:
df_old = pd.read_csv("WakaSwap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

df_timedelta.to_csv("WakaSwap export FTM.csv", index = False)

df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

In [ ]:
df_timedelta.to_csv("WakaSwap export FTM.csv", index = False)

In [ ]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[WakaSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

df_final_wakaswap_ftm = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_wakaswap_ftm.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_wakaswap_ftm.head(1)

In [ ]:
pd.set_option('display.max_columns', None)

df_final_combined = df_final_sushiswap_matic.append(df_final_quickswap)\
                                            .append(df_final_honeyswap_matic)\
                                            .append(df_final_wakaswap_ftm)\
                                            .append(df_final_spiritswap)\
                                            .append(df_final_spookyswap)\
                                            .append(df_final_sushiswap_ftm)\
                                            .sort_values(by = 'APR timedelta Liq + 200',
                                                         ascending = False)

df_final_combined[['APR timedelta',
                   'APR timedelta Liq + 200',
                   'APR timedelta Liq + 500',
                   'APR timedelta Liq + 1000',                   
                   'APR timedelta Liq + 2000',
                   'APR timedelta Liq - 200',
                   'APR timedelta Liq - 500',
                   'APR timedelta Liq - 1000',                   
                   'APR timedelta Liq - 2000',                   
                   'APR yday',
                   'APR yday Liq + 200',
                   'APR yday Liq + 500',
                   'APR yday Liq + 1000',                   
                   'APR yday Liq + 2000',
                   'APR yday Liq - 200',
                   'APR yday Liq - 500',
                   'APR yday Liq - 1000',                   
                   'APR yday Liq - 2000']] = df_final_combined[['APR timedelta',
                                                                'APR timedelta Liq + 200',
                                                               'APR timedelta Liq + 500',
                                                               'APR timedelta Liq + 1000',                   
                                                               'APR timedelta Liq + 2000',
                                                               'APR timedelta Liq - 200',
                                                               'APR timedelta Liq - 500',
                                                               'APR timedelta Liq - 1000',                   
                                                               'APR timedelta Liq - 2000',                       
                                                                'APR yday',
                                                                'APR yday Liq + 200',
                                                               'APR yday Liq + 500',
                                                               'APR yday Liq + 1000',                   
                                                               'APR yday Liq + 2000',
                                                               'APR yday Liq - 200',
                                                               'APR yday Liq - 500',
                                                               'APR yday Liq - 1000',                   
                                                               'APR yday Liq - 2000']].round(2)

df_final_combined[['Liquidity timedelta',
                   'Liquidity yday',
                   'Volume yday']] = df_final_combined[['Liquidity timedelta',
                                                        'Liquidity yday',
                                                        'Volume yday']].fillna(0).astype(int)

df_final_combined['APR timedelta'] = df_final_combined['APR timedelta'].round(1)

criteria = (df_final_combined['Volume yday'] > 0) & (df_final_combined['APR timedelta'] > 0)

df_final_combined.loc[criteria, :].dropna(subset = ['date'])